构建神经网络
========================

神经网络由对数据执行运算的层/模块组成。 torch.nn 命名空间提供了构建您自己的神经网络所需的所有构建块。PyTorch 中的每个模块都是 nn.Module 的子类。神经网络本身就是一个模块，由其他模块（层）组成。这种嵌套结构允许轻松构建和管理复杂的架构。

在以下部分中，我们将构建一个神经网络来对 FashionMNIST 数据集中的图像进行分类。


In [18]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

获取用于训练的设备
=======================

我们希望能够在 GPU 或 MPS 等硬件加速器上训练我们的模型（如果可用）。让我们检查一下 torch.cuda 或 torch.backends.mps 是否可用，否则我们将使用 CPU。


In [19]:
# 有啥用啥
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


定义类
================

我们通过子类化 nn.Module 来定义神经网络，并在 __init__ 中初始化神经网络层。每个 nn.Module 子类都在 forward 方法中实现对输入数据的操作。

. nn.Flatten() 的功能
将张量的形状展平为一维向量（从第二个维度开始）。
例如，输入形状是 
(
𝑁
,
𝐶
,
𝐻
,
𝑊
)
(N,C,H,W)，经过 Flatten() 后会变成 
(
𝑁
,
𝐶
×
𝐻
×
𝑊
)
(N,C×H×W)。
N 是批量大小（batch size）。
C 是通道数。
H, W 是高和宽。
只对非批量维度（即从第二维度开始）进行展平，批量维度 
𝑁
N 保持不变。


In [20]:
class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__() # 初始化 继承父类

        # 定义好类方法
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    # 前向传播（预测推理）的时候，使用类方法处理
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits



我们创建 NeuralNetwork 的实例，并将它移动到 device 上，然后打印它的结构。

In [21]:
model = NeuralNetwork().to(device)  # 模型移动到GPU
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


要使用模型，我们将输入数据传递给它。这将执行模型的 forward 方法，以及一些 后台操作。
不要直接调用 model.forward()！

在输入数据上调用模型将返回一个二维张量，其中 dim=0 对应于每个输出的 10 个原始预测值（每个类别一个），dim=1 对应于每个输出的各个值。我们通过将它传递给 nn.Softmax 模块的实例来获得预测概率。


In [22]:
X = torch.rand(1, 28, 28, device=device) # 数据构造在GPU
logits = model(X)
logits


tensor([[ 0.0149,  0.0087,  0.0222, -0.0285, -0.0839,  0.1577,  0.0261, -0.0170,
          0.0846, -0.0053]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [23]:
pred_probab = nn.Softmax(dim=1)(logits) # dim = 1 每行归一化，每行所有列sum = 1
pred_probab

tensor([[0.0995, 0.0989, 0.1002, 0.0953, 0.0901, 0.1148, 0.1006, 0.0964, 0.1067,
         0.0975]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [24]:
pred_probab = nn.Softmax(dim=1)(logits) # dim = 1 每行归一化，每行所有列sum = 1
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([5], device='cuda:0')


------------------------------------------------------------------------


模型层
============

让我们分解一下 FashionMNIST 模型中的层。为了说明这一点，我们将取一个大小为 28x28 的 3 张图像的样本小批量，看看它在通过网络时会发生什么。


In [25]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten
==========

我们初始化 nn.Flatten 层，将每个 2D 28x28 图像转换为 784 个像素值的连续数组（小批量维度（在 dim=0 处）保持不变）。


In [32]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

# 可以看到flatten时，batch的size是不变的

torch.Size([3, 784])


nn.Linear
=========

线性层 是一个模块，它使用其存储的权重和偏差对输入应用线性变换。


In [35]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

# 这里储存的权重和偏差，还没有经过训练，所以有可能是随机的

torch.Size([3, 20])


nn.ReLU
=======

非线性激活是模型的输入和输出之间创建复杂映射的关键。它们在线性变换之后应用，以引入非线性，帮助神经网络学习各种现象。

在这个模型中，我们在线性层之间使用了 nn.ReLU，但还有其他激活可以在模型中引入非线性。


In [37]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

# 非线性变换

Before ReLU: tensor([[0.0242, 0.1932, 0.0000, 0.2205, 0.4263, 0.0000, 0.3375, 0.0000, 0.0000,
         0.2893, 0.0000, 0.0000, 0.0000, 0.3121, 0.0000, 0.0000, 0.0000, 0.6705,
         0.3240, 0.5428],
        [0.0000, 0.1709, 0.0000, 0.1613, 0.6608, 0.0000, 0.2555, 0.0990, 0.1571,
         0.2042, 0.0000, 0.0000, 0.0000, 0.1893, 0.0965, 0.0000, 0.3434, 0.3892,
         0.2041, 0.0000],
        [0.0000, 0.0915, 0.0000, 0.0000, 0.5901, 0.0000, 0.2260, 0.0000, 0.0000,
         0.1314, 0.0034, 0.0000, 0.0000, 0.1036, 0.0000, 0.0000, 0.4024, 0.9076,
         0.3694, 0.0000]], grad_fn=<ReluBackward0>)


After ReLU: tensor([[0.0242, 0.1932, 0.0000, 0.2205, 0.4263, 0.0000, 0.3375, 0.0000, 0.0000,
         0.2893, 0.0000, 0.0000, 0.0000, 0.3121, 0.0000, 0.0000, 0.0000, 0.6705,
         0.3240, 0.5428],
        [0.0000, 0.1709, 0.0000, 0.1613, 0.6608, 0.0000, 0.2555, 0.0990, 0.1571,
         0.2042, 0.0000, 0.0000, 0.0000, 0.1893, 0.0965, 0.0000, 0.3434, 0.3892,
         0.2041, 0.0000],
       

nn.Sequential
=============

nn.Sequential 是模块的有序容器。数据按照定义的顺序依次通过所有模块。您可以使用顺序容器来快速构建网络，例如 seq_modules。


In [38]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

nn.Softmax
==========

神经网络的最后一个线性层返回logits - [-infty, infty] 范围内的原始值 - 这些值将传递给 nn.Softmax 模块。logits 被缩放到 [0, 1] 范围内的值，表示模型对每个类别的预测概率。 dim 参数指示沿哪个维度值必须加起来为 1。


In [30]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

模型参数
================

神经网络中的许多层都是参数化的，即具有在训练过程中优化的相关权重和偏差。子类化 nn.Module 会自动跟踪模型对象中定义的所有字段，并使用模型的 parameters() 或 named_parameters() 方法使所有参数可访问。

在这个示例中，我们遍历每个参数，并打印其大小及其值的预览。


In [31]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0251, -0.0059, -0.0127,  ...,  0.0322, -0.0211,  0.0357],
        [-0.0208, -0.0239, -0.0178,  ..., -0.0332, -0.0289, -0.0078]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0248, -0.0058], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0102,  0.0104, -0.0343,  ...,  0.0148, -0.0277, -0.0077],
        [-0.0177,  0.0083, -0.0131,  ...,  0.0007,  0.0072, -0.0194]],
       device='cuda:0', grad_fn=<Sl

------------------------------------------------------------------------
